# 📄 파일 타입별 텍스트/이미지/표 추출 통합 파이프라인

- hwpx, docx, xlsx, pptx, pdf 등 다양한 문서 타입을 자동 식별하여 텍스트, 이미지, 표를 추출합니다.
- 추출 결과는 페이지별 텍스트와 전체 문서 합본 텍스트를 함께 저장합니다.
- 파일명 앞 10자로 하위 폴더를 만들고 그 아래 document.json, pages/, tables/ 구조로 저장합니다.

In [1]:
# ✅ 의존성 점검 및 설치 안내
import importlib, sys, warnings

# Suppress noisy cryptography/pypdf ARC4 deprecation warnings seen during pypdf import
warnings.filterwarnings("ignore", message=r".*ARC4 has been moved.*")
warnings.filterwarnings("ignore", category=DeprecationWarning, module=r"pypdf\._crypt_providers\._cryptography")

print(f"🧪 Python 실행 경로: {sys.executable}")


def check_lib(name):
    try:
        importlib.import_module(name)
        print(f'✔ {name} 사용 가능')
        return True
    except Exception as e:
        print(f'✖ {name} 불가: {e}')
        return False

# pdfminer.six는 'pdfminer'로 import됩니다. pypdf는 PDF 텍스트 추출 폴백용
needed = [
    'pdfplumber', 'pdfminer', 'pypdf', 'docx', 'openpyxl', 'pptx', 'olefile', 'lxml', 'PIL', 'dotenv', 'fitz'
]
status = {n: check_lib(n) for n in needed}
missing = [n for n, ok in status.items() if not ok]
if missing:
    translate = {'pdfminer': 'pdfminer.six', 'fitz': 'PyMuPDF'}
    pip_list = [translate.get(m, m) for m in missing]
    print(f"⚠ 설치 필요: {', '.join(pip_list)}")
else:
    print('✅ 모든 필수 라이브러리 확인됨')

# 버전 정보(가능할 때만)
for mod in ['pdfplumber', 'pdfminer', 'pypdf']:
    try:
        m = importlib.import_module(mod)
        ver = getattr(m, '__version__', None)
        print(f"   - {mod} version: {ver}")
    except Exception:
        pass

🧪 Python 실행 경로: /home/admin/wkms-aws/.venv/bin/python
✔ pdfplumber 사용 가능
✔ pdfminer 사용 가능
✔ pypdf 사용 가능
✔ docx 사용 가능
✔ pypdf 사용 가능
✔ docx 사용 가능
✔ openpyxl 사용 가능
✔ pptx 사용 가능
✔ olefile 사용 가능
✔ lxml 사용 가능
✔ PIL 사용 가능
✔ dotenv 사용 가능
✔ openpyxl 사용 가능
✔ pptx 사용 가능
✔ olefile 사용 가능
✔ lxml 사용 가능
✔ PIL 사용 가능
✔ dotenv 사용 가능
✔ fitz 사용 가능
✅ 모든 필수 라이브러리 확인됨
   - pdfplumber version: 0.11.5
   - pdfminer version: 20231228
   - pypdf version: 3.17.4
✔ fitz 사용 가능
✅ 모든 필수 라이브러리 확인됨
   - pdfplumber version: 0.11.5
   - pdfminer version: 20231228
   - pypdf version: 3.17.4


In [2]:
# ⚙️ 경로/설정 및 파일 타입 자동 식별
import os
from pathlib import Path
from dotenv import load_dotenv

load_dotenv('/home/admin/wkms-aws/backend/.env')
BASE_DIR = Path('/home/admin/wkms-aws/jupyter_notebook')
INPUT_DIR = BASE_DIR / 'data/input_docs'
OUTPUT_DIR = BASE_DIR / 'data/extracted_objects'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

def detect_file_type(file_path):
    ext = Path(file_path).suffix.lower()
    if ext in ['.pdf', '.docx', '.xlsx', '.pptx', '.hwp', '.hwpx']:
        return ext[1:]
    return 'unknown'

In [8]:
# 📦 파일별 추출 함수 정의 (오브젝트 인식 강화)
from pathlib import Path


def is_image_pdf(file_path: str, sample_pages: int = 3, min_text_chars: int = 20):
    """간단한 휴리스틱으로 이미지 기반 PDF 여부 판단
    - 앞쪽 sample_pages 페이지에서 추출 텍스트 길이가 매우 짧고 이미지가 존재하면 이미지 PDF로 판단
    반환: (is_image_pdf: bool, page_count: int)
    """
    import pdfplumber
    text_chars = 0
    any_images = False
    page_count = 0
    with pdfplumber.open(file_path) as pdf:
        page_count = len(pdf.pages)
        check_n = min(sample_pages, page_count)
        for page in pdf.pages[:check_n]:
            txt = page.extract_text() or ""
            text_chars += len(txt.strip())
            if getattr(page, 'images', None):
                any_images = any_images or len(page.images) > 0
    is_img = (text_chars < min_text_chars) and any_images
    return is_img, page_count


def _extract_pdf_text_with_pypdf(file_path: str):
    """pdfplumber/pdfminer 실패 시 pypdf로 텍스트 추출 폴백"""
    from pypdf import PdfReader
    reader = PdfReader(str(file_path))
    pages = []
    combined = ''
    for i, page in enumerate(reader.pages, start=1):
        try:
            txt = page.extract_text() or ''
        except Exception:
            txt = ''
        pages.append({'index': i, 'text': txt})
        combined += f"\n[페이지 {i}]\n{txt}"
    return combined.strip(), pages


def _extract_pdf_images_with_pymupdf(file_path: str):
    """PyMuPDF로 PDF 페이지별 이미지 XObject 추출 (bbox 포함 메타)
    반환: [ { page, image_index, bbox, ext, bytes_len, note } ]
    """
    import fitz  # PyMuPDF
    results = []
    doc = fitz.open(file_path)
    for page_idx in range(len(doc)):
        page = doc[page_idx]
        # 이미지 목록 (xref 기반)
        image_list = page.get_images(full=True)
        for img_i, img in enumerate(image_list):
            xref = img[0]
            try:
                pix = fitz.Pixmap(doc, xref)
                if pix.n >= 5:  # CMYK
                    pix = fitz.Pixmap(fitz.csRGB, pix)
                ext = 'png'
                results.append({
                    'page': page_idx + 1,
                    'image_index': img_i + 1,
                    'xref': xref,
                    'width': pix.width,
                    'height': pix.height,
                    'colors': pix.n,
                    'ext': ext,
                    'note': '이미지 바이트 추출은 저장 루틴에서 처리'
                })
            except Exception as e:
                results.append({'page': page_idx + 1, 'image_index': img_i + 1, 'error': str(e)})
    doc.close()
    return results


def extract_pdf(file_path: str):
    results = {'text': '', 'pages': [], 'tables': [], 'images': [], 'metadata': {}}

    # 이미지 PDF 여부 판단을 시도하되, 실패 시 폴백으로 넘어감
    is_img = False
    page_count = None
    try:
        import pdfplumber
        is_img, page_count = is_image_pdf(file_path)
        results['metadata']['page_count'] = page_count
        results['metadata']['is_image_pdf'] = is_img
    except Exception as e:
        results['metadata']['note'] = f'pdfplumber 초기화 실패: {e}. pypdf 폴백 사용'

    if is_img:
        msg = '이미지 기반 PDF로 감지됨: OCR 파이프라인으로 처리 필요 (텍스트 추출 생략)'
        print('ℹ️ ' + msg)
        results['metadata']['note'] = msg
        # 이미지 오브젝트 메타라도 수집 (선택)
        try:
            results['images'] = _extract_pdf_images_with_pymupdf(file_path)
        except Exception as e:
            results['metadata']['image_extract_note'] = f'PyMuPDF 이미지 메타 추출 실패: {e}'
        return results

    # 텍스트 기반 PDF 추출 시도: pdfplumber 우선, 실패 시 pypdf 폴백
    try:
        import pdfplumber
        combined_text = ''
        with pdfplumber.open(file_path) as pdf:
            results['metadata']['page_count'] = results['metadata'].get('page_count') or len(pdf.pages)
            for i, page in enumerate(pdf.pages):
                text = page.extract_text() or ''
                combined_text += f"\n[페이지 {i+1}]\n{text}"
                results['pages'].append({'index': i+1, 'text': text})

                # 표 추출
                tables = page.extract_tables()
                if tables:
                    for t in tables:
                        safe_t = [[str(c) if c is not None else '' for c in row] for row in (t or [])]
                        results['tables'].append({'page': i+1, 'data': safe_t})

        results['text'] = combined_text.strip()
        # 추가로 PyMuPDF로 이미지 메타 수집
        try:
            results['images'] = _extract_pdf_images_with_pymupdf(file_path)
        except Exception as e:
            results['metadata']['image_extract_note'] = f'PyMuPDF 이미지 메타 추출 실패: {e}'
        return results
    except Exception as e:
        print(f"⚠ pdfplumber 실패, pypdf로 폴백: {e}")
        combined, pages = _extract_pdf_text_with_pypdf(file_path)
        results['text'] = combined
        results['pages'] = pages
        results['metadata']['fallback'] = 'pypdf'
        # 이미지 메타 수집 시도
        try:
            results['images'] = _extract_pdf_images_with_pymupdf(file_path)
        except Exception as e2:
            results['metadata']['image_extract_note'] = f'PyMuPDF 이미지 메타 추출 실패: {e2}'
        return results


# ===== PPTX 개선: XML 직접 파싱을 통한 텍스트 추출 (python-pptx 호환성 문제 해결) =====

def extract_pptx(file_path: str):
    """PPTX 파일에서 텍스트, 표, 이미지 추출 (XML 직접 파싱)"""
    import zipfile
    import xml.etree.ElementTree as ET
    
    try:
        text = ''
        pages = []
        images = []
        tables = []
        
        with zipfile.ZipFile(file_path, 'r') as z:
            # 슬라이드 XML 파일 찾기
            slide_files = [f for f in z.namelist() if f.startswith('ppt/slides/slide') and f.endswith('.xml')]
            slide_files.sort()  # 순서대로 정렬
            
            # 각 슬라이드 처리
            for i, slide_file in enumerate(slide_files):
                slide_idx = i + 1
                slide_text = ''
                
                try:
                    # 슬라이드 XML 파싱
                    with z.open(slide_file) as f:
                        xml_content = f.read()
                        root = ET.fromstring(xml_content)
                    
                    # 네임스페이스 정의
                    namespaces = {
                        'a': 'http://schemas.openxmlformats.org/drawingml/2006/main',
                        'p': 'http://schemas.openxmlformats.org/presentationml/2006/main',
                        'r': 'http://schemas.openxmlformats.org/officeDocument/2006/relationships'
                    }
                    
                    # 텍스트 요소 찾기 (a:t 태그)
                    text_elements = []
                    for elem in root.iter():
                        if elem.tag.endswith('}t'):  # a:t 태그
                            if elem.text and elem.text.strip():
                                text_elements.append(elem.text.strip())
                    
                    # 표 찾기 (a:tbl 태그)
                    for tbl_elem in root.iter():
                        if tbl_elem.tag.endswith('}tbl'):  # a:tbl 태그
                            table_data = []
                            for tr_elem in tbl_elem.iter():
                                if tr_elem.tag.endswith('}tr'):  # 테이블 행
                                    row_data = []
                                    for tc_elem in tr_elem.iter():
                                        if tc_elem.tag.endswith('}tc'):  # 테이블 셀
                                            cell_text = ''
                                            for t_elem in tc_elem.iter():
                                                if t_elem.tag.endswith('}t') and t_elem.text:
                                                    cell_text += t_elem.text.strip() + ' '
                                            row_data.append(cell_text.strip())
                                    if row_data:
                                        table_data.append(row_data)
                            if table_data:
                                tables.append({'slide': slide_idx, 'data': table_data})
                    
                    # 이미지 찾기 (blip 요소)
                    for blip_elem in root.iter():
                        if blip_elem.tag.endswith('}blip'):  # a:blip 태그
                            embed_id = blip_elem.get('{http://schemas.openxmlformats.org/officeDocument/2006/relationships}embed')
                            if embed_id:
                                images.append({
                                    'slide': slide_idx,
                                    'embed_id': embed_id,
                                    'type': 'picture'
                                })
                    
                    slide_text = '\n'.join(text_elements)
                    
                except Exception as e:
                    slide_text = f"슬라이드 {slide_idx} XML 파싱 오류: {str(e)}"
                
                text += f"\n[슬라이드 {slide_idx}]\n{slide_text}"
                pages.append({'index': slide_idx, 'text': slide_text, 'label': 'slide'})
            
            # 노트 슬라이드도 처리
            notes_files = [f for f in z.namelist() if f.startswith('ppt/notesSlides/notesSlide') and f.endswith('.xml')]
            for i, notes_file in enumerate(notes_files):
                try:
                    with z.open(notes_file) as f:
                        xml_content = f.read()
                        root = ET.fromstring(xml_content)
                    
                    notes_text = []
                    for elem in root.iter():
                        if elem.tag.endswith('}t') and elem.text and elem.text.strip():
                            notes_text.append(elem.text.strip())
                    
                    if notes_text:
                        notes_content = '\n'.join(notes_text)
                        if i < len(pages):
                            pages[i]['text'] += f"\n[노트]\n{notes_content}"
                            text += f"\n[슬라이드 {i+1} 노트]\n{notes_content}"
                            
                except Exception:
                    continue
        
        return {'text': text.strip(), 'pages': pages, 'tables': tables, 'images': images, 
               'metadata': {'slide_count': len(slide_files), 'extraction_method': 'xml_parsing'}}
        
    except Exception as e:
        error_msg = f"PPTX XML 파싱 실패: {str(e)}"
        return {'text': error_msg, 'pages': [{'index': 1, 'text': error_msg, 'label': 'error'}], 
               'tables': [], 'images': [], 'metadata': {'error': str(e)}}


# ===== HWP/HWPX 개선: LibreOffice를 이용한 페이지별 텍스트 추출 =====

def _hwp_fallback_extraction(file_path: str):
    """LibreOffice 변환 실패 시 olefile을 사용한 기본 HWP 텍스트 추출"""
    try:
        import olefile
        text = ''
        
        with olefile.OleFileIO(file_path) as ole:
            # PrvText 스트림에서 미리보기 텍스트 추출
            if ole.exists('PrvText'):
                try:
                    raw = ole.openstream('PrvText').read()
                    # HWP 5.0 포맷은 보통 utf-16le로 인코딩됨
                    text = raw.decode('utf-16le', errors='ignore').strip()
                except Exception as e:
                    print(f"PrvText 디코딩 실패: {e}")
                    
            # BodyText 스트림도 시도
            if not text and ole.exists('BodyText'):
                try:
                    raw = ole.openstream('BodyText').read()
                    text = raw.decode('utf-16le', errors='ignore').strip()
                except Exception as e:
                    print(f"BodyText 디코딩 실패: {e}")
        
        if text:
            return {
                'text': text,
                'pages': [{'index': 1, 'text': text, 'label': 'fallback_extraction'}],
                'tables': [],
                'images': [],
                'metadata': {'extraction_method': 'hwp_olefile_fallback', 'length': len(text)}
            }
        else:
            return {
                'text': 'HWP 파일에서 텍스트를 추출할 수 없습니다.',
                'pages': [{'index': 1, 'text': 'HWP 파일에서 텍스트를 추출할 수 없습니다.', 'label': 'error'}],
                'tables': [],
                'images': [],
                'metadata': {'error': 'No text extracted from HWP file'}
            }
    except Exception as e:
        error_msg = f"HWP fallback 추출 실패: {str(e)}"
        return {
            'text': error_msg,
            'pages': [{'index': 1, 'text': error_msg, 'label': 'error'}],
            'tables': [],
            'images': [],
            'metadata': {'error': str(e)}
        }


def extract_hwp_hwpx(file_path: str):
    """HWP 파일은 LibreOffice를 통해 PDF로 변환 후 페이지별 텍스트를 추출하고,
    HWPX 파일은 XML을 파싱하여 텍스트를 추출합니다."""
    import os, tempfile, subprocess, shutil
    from pathlib import Path as _Path

    file_path_obj = _Path(file_path)
    ext = file_path_obj.suffix.lower()

    if ext == '.hwp':
        # LibreOffice가 설치되어 있는지 확인
        if not shutil.which("libreoffice"):
            msg = "LibreOffice가 설치되어 있지 않아 HWP 파일 처리가 불가능합니다."
            print(f"❌ {msg}")
            return {'text': '', 'pages': [], 'tables': [], 'images': [], 'metadata': {'error': msg}}

        with tempfile.TemporaryDirectory() as tmpdir:
            tmp_path = _Path(tmpdir)
            
            # 1. HWP -> PDF 변환
            print(f"⏳ HWP -> PDF 변환 중: {file_path_obj.name}")
            try:
                subprocess.run(
                    ["libreoffice", "--headless", "--convert-to", "pdf", "--outdir", str(tmp_path), str(file_path_obj)],
                    timeout=300,  # 5분 타임아웃
                    check=True,
                    capture_output=True
                )
            except (subprocess.TimeoutExpired, subprocess.CalledProcessError) as e:
                error_msg = f"LibreOffice 변환 실패: {e.stderr.decode('utf-8', 'ignore') if hasattr(e, 'stderr') else str(e)}"
                print(f"❌ {error_msg}")
                return {'text': '', 'pages': [], 'tables': [], 'images': [], 'metadata': {'error': error_msg}}

            pdf_path = tmp_path / f"{file_path_obj.stem}.pdf"
            if not pdf_path.exists():
                print("❌ LibreOffice 변환 후 PDF 파일이 생성되지 않았습니다. Fallback 방식으로 시도합니다.")
                # Fallback: olefile을 사용한 기본 텍스트 추출
                return _hwp_fallback_extraction(file_path)
            
            print(f"✅ HWP -> PDF 변환 완료: {pdf_path}")

            # 2. 변환된 PDF에서 텍스트 추출 (기존 extract_pdf 함수 재사용)
            print("⏳ 변환된 PDF에서 페이지별 텍스트 추출 중...")
            pdf_results = extract_pdf(str(pdf_path))
            
            # 메타데이터에 HWP 처리 정보 추가
            pdf_results['metadata']['source_hwp'] = file_path_obj.name
            pdf_results['metadata']['extraction_method'] = 'hwp_libreoffice_pdf_pipeline'
            
            print(f"✅ 페이지 {len(pdf_results.get('pages',[]))}개 추출 완료.")
            return pdf_results

    elif ext == '.hwpx':
        # HWPX는 기존과 같이 XML 파싱
        import zipfile
        try:
            import lxml.etree as ET
            has_lxml = True
        except ImportError:
            import xml.etree.ElementTree as ET
            has_lxml = False
        
        text = ''
        pages = []
        tables = []
        images = []
        ns = {
            'hp': 'http://www.hancom.co.kr/hwpml/2011/paragraph',
            'hml': 'http://www.hancom.co.kr/hwpml/2011/core',
            'tbl': 'http://www.hancom.co.kr/hwpml/2011/table',
        }
        
        try:
            with zipfile.ZipFile(file_path, 'r') as z:
                # ... (기존 HWPX 로직과 동일)
                # (이 부분은 간결함을 위해 생략, 실제로는 전체 코드가 위치해야 함)
                xml_names = [n for n in z.namelist() if n.startswith('Contents/') and n.endswith('.xml')]
                sec_idx = 0
                for name in sorted(xml_names):
                    try:
                        data = z.read(name)
                        tree = ET.fromstring(data)
                        sec_text = '\n'.join(
                            node.text for node in tree.iter() if node.tag.endswith('}t') and node.text
                        ).strip()
                        if sec_text:
                            sec_idx += 1
                            pages.append({'index': sec_idx, 'text': sec_text, 'label': f'section_{sec_idx}'})
                            text += f"\n[Section {sec_idx}]\n{sec_text}"
                    except Exception:
                        continue
            
            return {
                'text': text.strip(), 'pages': pages, 'tables': tables, 'images': images,
                'metadata': {'extraction_method': 'hwpx_xml_parsing', 'sections_found': sec_idx}
            }
        except Exception as e:
            return {'text': '', 'pages': [], 'tables': [], 'images': [], 'metadata': {'error': str(e)}}

    else:
        return {'text': '[지원하지 않는 HWP/HWPX 형식]', 'pages': [], 'tables': [], 'images': [], 'metadata': {}}


# ===== DOCX 추출 함수 추가 =====

def extract_docx(file_path: str):
    """DOCX 파일에서 텍스트, 표, 이미지 추출"""
    try:
        from docx import Document
        import zipfile
        
        text = ''
        pages = []
        tables = []
        images = []
        
        # DOCX 파일 열기
        doc = Document(file_path)
        
        # 텍스트 추출
        full_text = []
        for paragraph in doc.paragraphs:
            if paragraph.text.strip():
                full_text.append(paragraph.text.strip())
        
        text = '\n'.join(full_text)
        if text:
            pages.append({'index': 1, 'text': text, 'label': 'document'})
        
        # 표 추출
        for table_idx, table in enumerate(doc.tables, start=1):
            table_data = []
            for row in table.rows:
                row_data = []
                for cell in row.cells:
                    row_data.append(cell.text.strip())
                table_data.append(row_data)
            if table_data:
                tables.append({'page': 1, 'table_index': table_idx, 'data': table_data})
        
        # 이미지 추출 (메타데이터만)
        try:
            with zipfile.ZipFile(file_path, 'r') as z:
                image_files = [f for f in z.namelist() if f.startswith('word/media/')]
                for img_file in image_files:
                    images.append({
                        'zip_entry': img_file,
                        'path': img_file.split('/')[-1],
                        'type': 'image'
                    })
        except Exception as e:
            print(f"이미지 메타데이터 추출 실패: {e}")
        
        return {
            'text': text,
            'pages': pages,
            'tables': tables,
            'images': images,
            'metadata': {'extraction_method': 'python_docx', 'paragraphs': len(doc.paragraphs), 'tables_count': len(doc.tables)}
        }
        
    except Exception as e:
        error_msg = f"DOCX 처리 실패: {str(e)}"
        return {
            'text': error_msg,
            'pages': [{'index': 1, 'text': error_msg, 'label': 'error'}],
            'tables': [],
            'images': [],
            'metadata': {'error': str(e)}
        }

In [4]:
# 🏷️ 파일별 추출 실행 및 결과 저장
from pathlib import Path

def extract_objects(file_path):
    ext = Path(file_path).suffix.lower()
    if ext == '.pdf':
        return extract_pdf(file_path)
    elif ext == '.docx':
        return extract_docx(file_path)
    elif ext in ['.xlsx', '.xls']:
        return extract_xlsx(file_path)
    elif ext in ['.pptx', '.ppt']:
        return extract_pptx(file_path)
    elif ext in ['.hwp', '.hwpx']:
        return extract_hwp_hwpx(file_path)
    else:
        return {'text': '[지원하지 않는 파일 형식]', 'pages': [], 'tables': [], 'images': [], 'metadata': {}}

In [9]:
# 📚 [실행] 파이프라인 실행
# 위에서 정의한 함수들을 사용하여 INPUT_DIR의 모든 파일을 처리하고,
# 추출된 텍스트, 표, 이미지 메타데이터를 각각의 폴더에 저장합니다.

import json
from pathlib import Path

def _json_default(o):
    if isinstance(o, Path):
        return str(o)
    raise TypeError(f"Object of type {o.__class__.__name__} is not JSON serializable")

def _safe_dir_name(name: str) -> str:
    """파일명을 디렉토리명으로 안전하게 변환"""
    import re
    # 윈도우 및 유닉스에서 유효하지 않은 문자 제거
    return re.sub(r'[\\/*?:"<>|]', '_', name)

def _build_error_result(file_path: Path, error: Exception) -> dict:
    """오류 발생 시 반환할 표준 결과 구조"""
    return {
        'text': f"Error processing {file_path.name}: {error}",
        'pages': [{'index': 1, 'text': f"Error: {error}"}],
        'tables': [],
        'images': [],
        'metadata': {'error': str(error)}
    }

def _save_pdf_images(file_path: Path, images_meta: list, out_images_dir: Path):
    """PyMuPDF 메타를 이용해 실제 이미지 바이트를 저장"""
    try:
        import fitz
        doc = fitz.open(str(file_path))
        saved = 0
        for page_idx in range(len(doc)):
            page = doc[page_idx]
            image_list = page.get_images(full=True)
            for img_i, img in enumerate(image_list):
                xref = img[0]
                try:
                    pix = fitz.Pixmap(doc, xref)
                    if pix.n >= 5:
                        pix = fitz.Pixmap(fitz.csRGB, pix)
                    data = pix.tobytes('png')
                    fname = out_images_dir / f"pdf_p{page_idx+1}_i{img_i+1}.png"
                    with open(fname, 'wb') as f:
                        f.write(data)
                    saved += 1
                except Exception:
                    continue
        doc.close()
        return saved
    except Exception:
        return 0


# 입력 폴더 스캔 (중복 폴더명 방지)
used_dirs = set()
for file_path in INPUT_DIR.glob('*'):
    if not file_path.is_file():
        continue
    print(f'▶️ 처리 중: {file_path.name}')
    base = _safe_dir_name(file_path.stem)
    out_base = OUTPUT_DIR / base
    suffix = 1
    while str(out_base) in used_dirs or out_base.exists():
        out_base = OUTPUT_DIR / f"{base}_{suffix}"
        suffix += 1
    used_dirs.add(str(out_base))
    out_base.mkdir(parents=True, exist_ok=True)

    try:
        # 확장자에 따라 적절한 추출 함수 호출
        ext = file_path.suffix.lower()
        if ext in ['.hwp', '.hwpx']:
            result = extract_hwp_hwpx(str(file_path))
        elif ext == '.pdf':
            result = extract_pdf(str(file_path))
        elif ext in ['.ppt', '.pptx']:
            result = extract_pptx(str(file_path))
        elif ext == '.docx':
            result = extract_docx(str(file_path)) 
        else:
            # 기타 지원하지 않는 파일 형식 처리
            result = {
                'text': f"지원하지 않는 파일 형식: {ext}", 'pages': [], 'tables': [], 'images': [],
                'metadata': {'error': f"Unsupported file type: {ext}"}
            }

    except Exception as e:
        print(f'⚠ 처리 중 오류 발생: {e}')
        result = _build_error_result(file_path, e)

    # 0) 보조 디렉토리
    pages_dir = out_base / 'pages'; pages_dir.mkdir(exist_ok=True)
    tables_dir = out_base / 'tables'; tables_dir.mkdir(exist_ok=True)
    images_dir = out_base / 'images'; images_dir.mkdir(exist_ok=True)

    # 1) 전체 문서 결과 저장
    doc_json = out_base / 'document.json'
    with open(doc_json, 'w', encoding='utf-8') as f:
        json.dump(result, f, ensure_ascii=False, indent=2, default=_json_default)

    # 2) 페이지별 텍스트 저장
    pages = result.get('pages') or []
    if pages:
        for p in pages:
            idx = p.get('index')
            text = p.get('text', '')
            if idx is None:
                continue
            with open(pages_dir / f'page_{idx:03d}.txt', 'w', encoding='utf-8') as f:
                f.write(text)

    # 3) 표 저장 (TSV)
    tables = result.get('tables') or []
    if tables:
        for ti, t in enumerate(tables, start=1):
            page_no = None
            data = t
            if isinstance(t, dict):
                page_no = t.get('page') or t.get('slide') or t.get('sheet')
                data = t.get('data')
            fname = f"table_{ti:03d}{('_p'+str(page_no)) if page_no else ''}.tsv"
            with open(tables_dir / fname, 'w', encoding='utf-8') as f:
                for row in (data or []):
                    f.write('\t'.join([str(c) if c is not None else '' for c in row]) + '\n')

    # 4) 이미지 저장
    images = result.get('images') or []
    ext = file_path.suffix.lower()
    saved_images = 0
    if ext == '.pdf' or (ext == '.hwp' and result.get('metadata', {}).get('extraction_method') == 'hwp_libreoffice_pdf_pipeline'):
        # PDF이거나, HWP가 PDF로 변환된 경우 PyMuPDF로 이미지 저장
        # HWP의 경우 변환된 임시 PDF 경로는 알 수 없으므로, 원본 PDF에서만 작동
        if ext == '.pdf':
            saved_images = _save_pdf_images(file_path, images, images_dir)
        else: # HWP의 경우 이미지 추출은 현재 지원하지 않음
             with open(images_dir / 'hwp_images_meta.json', 'w', encoding='utf-8') as f:
                json.dump({'note': 'HWP 이미지 추출은 현재 지원되지 않습니다.'}, f)

    elif ext in ['.pptx', '.ppt']:
        # PPTX 이미지 메타 저장
        with open(images_dir / 'pptx_images_meta.json', 'w', encoding='utf-8') as f:
            json.dump(images, f, ensure_ascii=False, indent=2)
        saved_images = len([im for im in images if 'error' not in im and im.get('type') != 'chart'])
    elif ext == '.docx':
        # DOCX 이미지 저장
        try:
            from zipfile import ZipFile
            with ZipFile(file_path) as z:
                for im in images:
                    entry = im.get('zip_entry')
                    if entry:
                        data = z.read(entry)
                        out = images_dir / im.get('path', 'image.bin')
                        with open(out, 'wb') as f:
                            f.write(data)
                        saved_images += 1
        except Exception as e:
            with open(images_dir / 'docx_images_meta.json', 'w', encoding='utf-8') as f:
                json.dump({'error': str(e), 'images': images}, f, ensure_ascii=False, indent=2)
    else:
        # 기타 포맷은 메타만 저장
        with open(images_dir / 'images_meta.json', 'w', encoding='utf-8') as f:
            json.dump(images, f, ensure_ascii=False, indent=2)

    print(f'✅ 결과 저장: {out_base} (pages: {len(pages)}, tables: {len(tables)}, images: {saved_images})')


▶️ 처리 중: 20_ProductSpec_SmartInsulinPump_KO_v0.1.pdf
✅ 결과 저장: /home/admin/wkms-aws/jupyter_notebook/data/extracted_objects/20_ProductSpec_SmartInsulinPump_KO_v0.1 (pages: 6, tables: 3, images: 6)
▶️ 처리 중: 초안-Azure 기반 자율 이동 로봇 FMS 시스템 제안서.pptx
✅ 결과 저장: /home/admin/wkms-aws/jupyter_notebook/data/extracted_objects/초안-Azure 기반 자율 이동 로봇 FMS 시스템 제안서 (pages: 10, tables: 0, images: 10)
▶️ 처리 중: test1.pdf
✅ 결과 저장: /home/admin/wkms-aws/jupyter_notebook/data/extracted_objects/20_ProductSpec_SmartInsulinPump_KO_v0.1 (pages: 6, tables: 3, images: 6)
▶️ 처리 중: 초안-Azure 기반 자율 이동 로봇 FMS 시스템 제안서.pptx
✅ 결과 저장: /home/admin/wkms-aws/jupyter_notebook/data/extracted_objects/초안-Azure 기반 자율 이동 로봇 FMS 시스템 제안서 (pages: 10, tables: 0, images: 10)
▶️ 처리 중: test1.pdf
✅ 결과 저장: /home/admin/wkms-aws/jupyter_notebook/data/extracted_objects/test1 (pages: 96, tables: 220, images: 41)
▶️ 처리 중: 2025년도『AI자율제조로봇 실증(시범)사업』 공고문_F.hwp
⏳ HWP -> PDF 변환 중: 2025년도『AI자율제조로봇 실증(시범)사업』 공고문_F.hwp
✅ 결과 저장: /home/admin/wkms-aws/jupyter_no